## Imports

In [1]:
import os

## Generate Slice Data

In [13]:
DATA_DIR = os.path.join("..","..","data")
ORIGINAL_DATA_PATH = os.path.join(DATA_DIR,"BrainTriage")
SLICED_DATA_DIR = os.path.join(DATA_DIR, "sliced-data")
TRAIN_PATH = os.path.join(SLICED_DATA_DIR, "train")
TEST_PATH = os.path.join(SLICED_DATA_DIR, "test")
DATA_SPLIT_DIR = os.path.join(DATA_DIR, "data-split")
print(DATA_DIR, ORIGINAL_DATA_PATH, SLICED_DATA_DIR, TRAIN_PATH, TEST_PATH, DATA_SPLIT_DIR, sep="\n")

../../data
../../data/BrainTriage
../../data/sliced-data
../../data/sliced-data/train
../../data/sliced-data/test
../../data/data-split


In [14]:
if not os.path.exists(TRAIN_PATH):
    !python3 dataset/create_slices.py -d $ORIGINAL_DATA_PATH -o $SLICED_DATA_DIR --train
if not os.path.exists(TEST_PATH):
    !python3 dataset/create_slices.py -d $ORIGINAL_DATA_PATH -o $SLICED_DATA_DIR --test

## Create Train/Validation Split

In [15]:
K = 10

In [16]:
if not os.path.exists(DATA_SPLIT_DIR):
    !python3 dataset/create_data_split.py -k $K -d $TRAIN_PATH -ds $DATA_SPLIT_DIR

1000


## Train ResNet34

In [17]:
RESNET_TYPE = "resnet34"
SEED = 420
LR = 0.0001
EPOCHS = 20
BATCH_SIZE = 16
MODEL_DIR = "../../models"

In [7]:
!python3 train/train-resnet.py $RESNET_TYPE -s $SEED -lr $LR -e $EPOCHS -b $BATCH_SIZE -m \
                               $MODEL_DIR -d $TRAIN_PATH -ds $DATA_SPLIT_DIR -ts 0 32 --tuple 

wandb: Appending key for api.wandb.ai to your netrc file: /Users/fbergh/.netrc
Successfully logged in to Weights & Biases!
Traceback (most recent call last):
  File "train/train-resnet.py", line 74, in <module>
    train_df = pd.read_csv(os.path.join(args.ds_dir, "train_df.csv"), names=["patient_nr", "slice_nr", "class"] ).sample(frac=1).reset_index(drop=True)
  File "/Users/fbergh/Library/Python/3.7/lib/python/site-packages/pandas/io/parsers.py", line 676, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/Users/fbergh/Library/Python/3.7/lib/python/site-packages/pandas/io/parsers.py", line 448, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "/Users/fbergh/Library/Python/3.7/lib/python/site-packages/pandas/io/parsers.py", line 880, in __init__
    self._make_engine(self.engine)
  File "/Users/fbergh/Library/Python/3.7/lib/python/site-packages/pandas/io/parsers.py", line 1114, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "

## Train LSTM (Freeze ResNet34)